In [1]:
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
from tqdm import tqdm

from src.models.geom_bm import GBMStepModel
from src.data_feeder.sim_data_feeder import SimDataFeeder
from src.timers import AcceleratedTimer, DeltaTimer
from src.data_loaders import LazyLoader
from src.markets import SimKalshiMarket
from src.backtester.linked_feeders import FeederCreator
from src.agents.hedging_agent import HedgingAgent
from src.exceptions import SimFinished

In [2]:
def get_terminal_port_values(max_under_pos: float, min_tte_hedge: float) -> list:
    """evaluate hedge strategy with the following two hyperparameters
    Args:
        max_under_pos (float): maximum magnitude of delta hedge in shares
        min_tte_hedge (float): will not rebalance hedge after this expiration time
    """
    end_values = []
    for deriv_feeder, under_feeder, timer, meta_data in FeederCreator.iterate(timedelta=60):

        #  looping through contracts
        deriv_feeder.start()
        under_feeder.start()
        agent = HedgingAgent(deriv_feeder, under_feeder, timer, meta_data['strike'], max_under_pos=max_under_pos, min_tte_hedge=min_tte_hedge)

        while True:
            timer.cycle()

            try:
                agent.consume()

            except SimFinished:
                # print(f"finished {data_points_count} {meta_data['strike']} {meta_data['date']}")
                net_end = agent.reconcile_hedge(meta_data['terminal_u_price']) + meta_data['outcome']
                end_values.append(net_end)
                break
    
    return end_values

        
        

In [3]:
results = []

In [ ]:
# backtest
max_max_under_pos = .0015
max_min_tte_hedge = .7
samples = 12
sampled_no_hedge = False
for max_under_pos in (np.linspace(0, max_max_under_pos, samples)):
    for min_tte_hedge in tqdm(np.linspace(0, max_min_tte_hedge, samples)):
        if max_under_pos == 0:
            if not sampled_no_hedge:
                sampled_no_hedge = True
            else:
                continue

        end_values = get_terminal_port_values(max_under_pos=max_under_pos, min_tte_hedge=min_tte_hedge)
        mean = np.mean(end_values)
        var = np.var(end_values)
        row = {
            "max_under_pos": max_under_pos,
            "min_tte_hedge": min_tte_hedge,
            "mean": mean,
            "var": var,
        }
        
        results.append(row)




In [ ]:
# save results
res_df = pd.DataFrame(results)
res_df.to_csv("hedge_agent_res.csv")

In [ ]:
# printing results
am = res_df['var'].argmin()
min_var = res_df.iloc[am]
no_hedge = res_df.iloc[0]
print("Variance With No Hedge")
print(no_hedge, "\n\n")

print("Variance With Delta Hedge")
print(min_var, "\n\n")

Variance With No Hedge
max_under_pos    0.000000
min_tte_hedge    0.000000
mean             0.491124
var              0.249921
Name: 0, dtype: float64 


Variance With Delta Hedge
max_under_pos    0.000682
min_tte_hedge    0.000000
mean             0.495915
var              0.155525
Name: 49, dtype: float64 


